In [12]:
import numpy as np
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

In [13]:
def boxes_iou(boxes1, boxes2):
    #锚框与真实框面积之和
    area_func = lambda w, h: abs(w * h)
    area1 = area_func(boxes1[:, 0] - boxes1[:, 2], boxes1[:, 1] - boxes1[:, 3])
    area2 = area_func(boxes2[:, 0] - boxes2[:, 2], boxes2[:, 1] - boxes2[:, 3])
    area_union = area1[:, None] + area2
    #锚框与真实框之间的交集
    left_upper = np.maximum(boxes1[:, None, :2], boxes2[:, :2])
    right_bottom = np.minimum(boxes1[:, None, -2:], boxes2[:, -2:])
    weight_height = np.clip(right_bottom - left_upper, a_min=0, a_max=None)
    area_inter = weight_height[:, :, 0] * weight_height[:, :, 1]
    #计算交并比
    iou = area_inter / (area_union - area_inter)
    return iou

In [14]:
def boxes_assign(anchor_boxes, bound_boxes, threshold=0.5):
    anchor_num, bound_num = anchor_boxes.shape[0], bound_boxes.shape[0]
    iou = boxes_iou(anchor_boxes, bound_boxes)
    values = np.amax(iou, axis=1)
    indices = np.argmax(iou, axis=1)
    anchor_id = np.where(values >= threshold)
    bound_id = indices[values >= threshold]
    anchor_bound = np.full(anchor_num, fill_value=-1)
    anchor_bound[anchor_id] = bound_id
    return anchor_bound

In [15]:
boxes1=np.random.randint(100, 500, size=(5, 4))
boxes2=np.random.randint(200, 400, size=(4, 4))
iou = boxes_iou(boxes1, boxes2)
boxes_assign(boxes1, boxes2)

array([-1, -1, -1, -1, -1])

In [16]:
if __name__ == "__main__":
    path = Path.cwd()
    imgs = list(path.rglob("dog?.jpg"))
    boxes = np.array([[200, 200, 400, 400], [300, 300, 600, 600]])
    x1 = np.max(boxes[:, 0])
    y1 = np.max(boxes[:, 1])
    x2 = np.min(boxes[:, 2])
    y2 = np.min(boxes[:, 3])
    area_inter = (x2 - x1 + 1) * (y2 - y1 + 1)
    area1 = (boxes[0, 2] - boxes[0, 0] + 1) * (boxes[0, 3] - boxes[0, 1] + 1)
    area2 = (boxes[1, 2] - boxes[1, 0] + 1) * (boxes[1, 3] - boxes[1, 1] + 1)
    iou = area_inter / (area1 + area2 - area_inter)

    with Image.open(imgs[0]) as img:
        draw = ImageDraw.Draw(img)
        print(img.size)
        draw.rectangle((x1, y1, x2, y2), "yellow", "blue")
        draw.rectangle(tuple(boxes[0]), outline="white", width=2)
        draw.rectangle(tuple(boxes[1]), outline="white", width=2)
        draw.text((x1, y1), f"iou:{iou:.2%}", fill="green", align="left")
        img.show()

(1280, 1280)
